In [1]:
import sys

In [3]:
%%bash

pip uninstall fma_prep -y
pip install --no-cache-dir . > /dev/null


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from fma_prep import run


2024-09-11 10:59:08.673166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 10:59:08.687521: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 10:59:08.691865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 10:59:08.702608: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 10:59:09.583436: W tensorflow/comp

In [6]:
# Defina os argumentos como se fossem passados pela linha de comando
sys.argv = [
    'fma_prep.py',  # Nome do script (pode ser qualquer string)
    '--input_path', '/home/bruno/storage/data/fma/fma-large-by-effnet-discogs',
    '--output_path', '/home/bruno/storage/data/fma',
    '--top_genres', 'Rock', 'Electronic', 
    '--sequence_size', '1280',
    '--train_id', 'rock_electronic',
    '--sample_size', '1'
]

In [7]:
# Chama a função run() com os argumentos definidos
run()

Prepraring paths.
Using top genres list. ['Rock', 'Electronic']
Crerating labels structures.
Spliting dataset in train/test/val.


100%|██████████| 258/258 [00:00<00:00, 1906.61it/s]
2024-09-11 11:08:14.495179: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


0 2006 /home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/val/0000000000.tfrecord
1/1 batchs / 51200 processed
1/1 batchs / 2006 processed
0 4814 /home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/test/0000000000.tfrecord
1/1 batchs / 51200 processed
1/1 batchs / 4814 processed
0 18209 /home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/train/0000000000.tfrecord
1/1 batchs / 51200 processed
1/1 batchs / 18209 processed
0 2006 /home/bruno/storage/data/fma/trains/rock_electronic/torch/val/0000000000.pt
1/1 batches / 51200 processed
1/1 batches / 2006 processed
0 4814 /home/bruno/storage/data/fma/trains/rock_electronic/torch/test/0000000000.pt
1/1 batches / 51200 processed
1/1 batches / 4814 processed
0 18209 /home/bruno/storage/data/fma/trains/rock_electronic/torch/train/0000000000.pt
1/1 batches / 51200 processed
1/1 batches / 18209 processed
